# Funnel Chart - Conversion Process Visualization

**Use Case**: Show step-by-step conversion rates, sales pipelines, user journey analysis, process efficiency

This notebook demonstrates how to create effective funnel charts for visualizing conversion processes and identifying bottlenecks.


In [ ]:
# Import necessary libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.graph_objects as go
import plotly.express as px
from matplotlib.patches import Polygon
import warnings
warnings.filterwarnings('ignore')

# Set style for better-looking plots
plt.style.use('seaborn-v0_8-whitegrid')
sns.set_palette("Set2")

# Set random seed for reproducibility
np.random.seed(42)

print("Funnel chart visualization libraries loaded!")


In [ ]:
# Create sample datasets for funnel charts
# 1. E-commerce Sales Funnel
ecommerce_funnel = pd.DataFrame({
    'Stage': ['Website Visitors', 'Product Views', 'Add to Cart', 'Checkout Started', 
              'Payment Info', 'Purchase Complete'],
    'Count': [100000, 45000, 18000, 12000, 8500, 6800],
    'Category': 'E-commerce'
})

# Calculate conversion rates
ecommerce_funnel['Conversion_Rate'] = (ecommerce_funnel['Count'] / ecommerce_funnel['Count'].iloc[0] * 100).round(2)
ecommerce_funnel['Step_Conversion'] = (ecommerce_funnel['Count'] / ecommerce_funnel['Count'].shift(1) * 100).round(2)
ecommerce_funnel.loc[0, 'Step_Conversion'] = 100.00

# 2. SaaS Lead Generation Funnel
saas_funnel = pd.DataFrame({
    'Stage': ['Marketing Qualified Leads', 'Sales Qualified Leads', 'Demos Scheduled', 
              'Proposals Sent', 'Negotiations', 'Closed Won'],
    'Count': [5000, 2800, 1600, 900, 600, 350],
    'Category': 'SaaS'
})

saas_funnel['Conversion_Rate'] = (saas_funnel['Count'] / saas_funnel['Count'].iloc[0] * 100).round(2)
saas_funnel['Step_Conversion'] = (saas_funnel['Count'] / saas_funnel['Count'].shift(1) * 100).round(2)
saas_funnel.loc[0, 'Step_Conversion'] = 100.00

# 3. Mobile App User Journey
app_funnel = pd.DataFrame({
    'Stage': ['App Downloads', 'Account Created', 'Tutorial Completed', 
              'First Action', 'Weekly Active', 'Premium Upgrade'],
    'Count': [80000, 52000, 38000, 28000, 15000, 2100],
    'Category': 'Mobile App'
})

app_funnel['Conversion_Rate'] = (app_funnel['Count'] / app_funnel['Count'].iloc[0] * 100).round(2)
app_funnel['Step_Conversion'] = (app_funnel['Count'] / app_funnel['Count'].shift(1) * 100).round(2)
app_funnel.loc[0, 'Step_Conversion'] = 100.00

# 4. Recruitment Process Funnel
recruitment_funnel = pd.DataFrame({
    'Stage': ['Applications Received', 'Resume Screening', 'Phone Interview', 
              'Technical Assessment', 'Final Interview', 'Job Offers', 'Accepted Offers'],
    'Count': [2500, 800, 450, 200, 120, 85, 68],
    'Category': 'Recruitment'
})

recruitment_funnel['Conversion_Rate'] = (recruitment_funnel['Count'] / recruitment_funnel['Count'].iloc[0] * 100).round(2)
recruitment_funnel['Step_Conversion'] = (recruitment_funnel['Count'] / recruitment_funnel['Count'].shift(1) * 100).round(2)
recruitment_funnel.loc[0, 'Step_Conversion'] = 100.00

# 5. Healthcare Patient Journey
healthcare_funnel = pd.DataFrame({
    'Stage': ['Symptoms Research', 'Doctor Search', 'Appointment Booked', 
              'Consultation', 'Treatment Plan', 'Treatment Completed'],
    'Count': [10000, 6500, 4200, 3800, 3200, 2650],
    'Category': 'Healthcare'
})

healthcare_funnel['Conversion_Rate'] = (healthcare_funnel['Count'] / healthcare_funnel['Count'].iloc[0] * 100).round(2)
healthcare_funnel['Step_Conversion'] = (healthcare_funnel['Count'] / healthcare_funnel['Count'].shift(1) * 100).round(2)
healthcare_funnel.loc[0, 'Step_Conversion'] = 100.00

# 6. Marketing Campaign Funnel (with multiple campaigns)
campaigns_data = []
campaign_names = ['Social Media', 'Email Marketing', 'Google Ads', 'Content Marketing', 'Referral Program']

for campaign in campaign_names:
    # Simulate different conversion patterns
    if campaign == 'Social Media':
        base_multiplier = 1.2
        conversion_pattern = [1.0, 0.15, 0.08, 0.05, 0.03, 0.018]
    elif campaign == 'Email Marketing':
        base_multiplier = 0.8
        conversion_pattern = [1.0, 0.25, 0.12, 0.08, 0.055, 0.035]
    elif campaign == 'Google Ads':
        base_multiplier = 1.5
        conversion_pattern = [1.0, 0.18, 0.09, 0.06, 0.04, 0.025]
    elif campaign == 'Content Marketing':
        base_multiplier = 0.6
        conversion_pattern = [1.0, 0.35, 0.18, 0.12, 0.08, 0.055]
    else:  # Referral Program
        base_multiplier = 0.4
        conversion_pattern = [1.0, 0.45, 0.25, 0.18, 0.12, 0.09]
    
    base_count = int(50000 * base_multiplier)
    counts = [int(base_count * rate) for rate in conversion_pattern]
    
    for i, (stage, count) in enumerate(zip(['Impressions', 'Clicks', 'Leads', 'Qualified', 'Opportunities', 'Customers'], counts)):
        campaigns_data.append({
            'Campaign': campaign,
            'Stage': stage,
            'Count': count,
            'Stage_Order': i
        })

campaigns_df = pd.DataFrame(campaigns_data)

print("Sample funnel chart datasets created:")
print(f"E-commerce Funnel: {len(ecommerce_funnel)} stages")
print(f"SaaS Lead Funnel: {len(saas_funnel)} stages")
print(f"Mobile App Funnel: {len(app_funnel)} stages")
print(f"Recruitment Funnel: {len(recruitment_funnel)} stages")
print(f"Healthcare Funnel: {len(healthcare_funnel)} stages")
print(f"Marketing Campaigns: {len(campaigns_df)} records across 5 campaigns")

# Display sample data
print(f"\nSample E-commerce Funnel:")
print(ecommerce_funnel[['Stage', 'Count', 'Conversion_Rate', 'Step_Conversion']])


In [ ]:
# Function to create funnel charts
def create_funnel_chart(df, title, ax, show_percentages=True, color_scheme='viridis'):
    """Create a funnel chart"""
    
    stages = df['Stage'].tolist()
    counts = df['Count'].tolist()
    
    # Normalize widths for funnel shape
    max_count = max(counts)
    widths = [count / max_count for count in counts]
    
    # Create funnel segments
    height_per_stage = 0.8 / len(stages)
    colors = plt.cm.get_cmap(color_scheme)(np.linspace(0.2, 0.8, len(stages)))
    
    for i, (stage, count, width) in enumerate(zip(stages, counts, widths)):
        # Calculate trapezoid coordinates
        y_bottom = i * height_per_stage
        y_top = (i + 1) * height_per_stage
        
        # Current and next width (for trapezoid shape)
        current_width = width
        next_width = widths[i + 1] if i < len(widths) - 1 else width
        
        # Create trapezoid
        left_bottom = 0.5 - current_width / 2
        right_bottom = 0.5 + current_width / 2
        left_top = 0.5 - next_width / 2
        right_top = 0.5 + next_width / 2
        
        if i == len(stages) - 1:  # Last segment - rectangle
            left_top = left_bottom
            right_top = right_bottom
        
        # Create polygon
        trapezoid = Polygon([(left_bottom, y_bottom), (right_bottom, y_bottom),
                           (right_top, y_top), (left_top, y_top)],
                          facecolor=colors[i], edgecolor='white', linewidth=2, alpha=0.8)
        ax.add_patch(trapezoid)
        
        # Add labels
        center_x = 0.5
        center_y = y_bottom + height_per_stage / 2
        
        # Stage name
        ax.text(center_x, center_y + 0.01, stage, ha='center', va='center', 
               fontweight='bold', fontsize=10, color='white')
        
        # Count and percentage
        if show_percentages:
            conversion_rate = df['Conversion_Rate'].iloc[i]
            step_conversion = df['Step_Conversion'].iloc[i]
            ax.text(center_x, center_y - 0.02, f'{count:,}\n({conversion_rate}% total)', 
                   ha='center', va='center', fontsize=9, color='white')
            
            # Step conversion rate (except for first stage)
            if i > 0:
                ax.text(1.05, center_y, f'{step_conversion}%', ha='left', va='center', 
                       fontsize=9, fontweight='bold', color=colors[i])
    
    ax.set_xlim(0, 1.3)
    ax.set_ylim(0, len(stages) * height_per_stage)
    ax.set_aspect('equal')
    ax.axis('off')
    ax.set_title(title, fontweight='bold', pad=20)
    
    return ax

# Create basic funnel charts
fig, axes = plt.subplots(2, 2, figsize=(20, 16))
fig.suptitle('Funnel Chart Visualizations - Conversion Process Analysis', fontsize=16, fontweight='bold')

# 1. E-commerce Sales Funnel
ax1 = axes[0, 0]
create_funnel_chart(ecommerce_funnel, 'E-commerce Sales Funnel', ax1, color_scheme='Blues')

# 2. SaaS Lead Generation Funnel
ax2 = axes[0, 1]
create_funnel_chart(saas_funnel, 'SaaS Lead Generation Funnel', ax2, color_scheme='Greens')

# 3. Mobile App User Journey
ax3 = axes[1, 0]
create_funnel_chart(app_funnel, 'Mobile App User Journey', ax3, color_scheme='Purples')

# 4. Recruitment Process Funnel
ax4 = axes[1, 1]
create_funnel_chart(recruitment_funnel, 'Recruitment Process Funnel', ax4, color_scheme='Oranges')

plt.tight_layout()
plt.show()


In [ ]:
# Advanced funnel techniques
fig, axes = plt.subplots(2, 2, figsize=(20, 16))
fig.suptitle('Advanced Funnel Chart Techniques', fontsize=16, fontweight='bold')

# 1. Multi-Campaign Funnel Comparison
ax1 = axes[0, 0]

# Create stacked bar chart for campaign comparison
campaigns_pivot = campaigns_df.pivot(index='Stage_Order', columns='Campaign', values='Count')
stages_order = ['Impressions', 'Clicks', 'Leads', 'Qualified', 'Opportunities', 'Customers']

# Calculate conversion rates for each campaign
for campaign in campaign_names:
    campaign_data = campaigns_df[campaigns_df['Campaign'] == campaign]
    impressions = campaign_data[campaign_data['Stage'] == 'Impressions']['Count'].iloc[0]
    customers = campaign_data[campaign_data['Stage'] == 'Customers']['Count'].iloc[0]
    overall_conversion = (customers / impressions) * 100
    
    # Plot campaign funnel
    campaign_counts = []
    for stage in stages_order:
        count = campaign_data[campaign_data['Stage'] == stage]['Count'].iloc[0]
        campaign_counts.append(count)
    
    # Normalize for comparison
    normalized_counts = [count / campaign_counts[0] for count in campaign_counts]
    x_pos = np.arange(len(stages_order))
    ax1.plot(x_pos, normalized_counts, marker='o', linewidth=2, label=f'{campaign} ({overall_conversion:.2f}%)')

ax1.set_xlabel('Funnel Stage')
ax1.set_ylabel('Conversion Rate (Normalized)')
ax1.set_title('Multi-Campaign Funnel Comparison\n(Normalized to Starting Point)')
ax1.set_xticks(x_pos)
ax1.set_xticklabels(stages_order, rotation=45, ha='right')
ax1.legend()
ax1.grid(True, alpha=0.3)

# 2. Funnel with Dropout Analysis
ax2 = axes[0, 1]

# Calculate dropouts for e-commerce funnel
ecommerce_analysis = ecommerce_funnel.copy()
ecommerce_analysis['Dropout'] = ecommerce_analysis['Count'].diff(-1).fillna(0)
ecommerce_analysis['Dropout_Rate'] = (ecommerce_analysis['Dropout'] / ecommerce_analysis['Count'] * 100).round(2)

# Create horizontal bar chart showing dropouts
stages = ecommerce_analysis['Stage'][:-1]  # Exclude last stage
dropouts = ecommerce_analysis['Dropout'][:-1]
dropout_rates = ecommerce_analysis['Dropout_Rate'][:-1]

y_pos = np.arange(len(stages))
bars = ax2.barh(y_pos, dropouts, color='red', alpha=0.7)

# Add dropout rate labels
for i, (bar, rate) in enumerate(zip(bars, dropout_rates)):
    width = bar.get_width()
    ax2.text(width + max(dropouts) * 0.01, bar.get_y() + bar.get_height()/2, 
            f'{rate:.1f}%', ha='left', va='center', fontweight='bold')

ax2.set_yticks(y_pos)
ax2.set_yticklabels(stages)
ax2.set_xlabel('Number of Dropouts')
ax2.set_title('E-commerce Funnel Dropout Analysis\n(Where Users Leave)')
ax2.grid(True, alpha=0.3, axis='x')

# 3. Time-based Funnel Analysis
ax3 = axes[1, 0]

# Simulate time-based conversion data
months = ['Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun']
monthly_conversions = {
    'Leads': [1000, 1200, 1100, 1300, 1400, 1350],
    'Qualified': [300, 360, 330, 390, 420, 405],
    'Opportunities': [150, 180, 165, 195, 210, 203],
    'Customers': [45, 54, 50, 59, 63, 61]
}

monthly_df = pd.DataFrame(monthly_conversions, index=months)

# Calculate monthly conversion rates
for month in months:
    monthly_conversion_rate = (monthly_df.loc[month, 'Customers'] / monthly_df.loc[month, 'Leads']) * 100
    ax3.plot(monthly_df.columns, monthly_df.loc[month], marker='o', linewidth=2, 
            label=f'{month} ({monthly_conversion_rate:.1f}%)')

ax3.set_xlabel('Funnel Stage')
ax3.set_ylabel('Count')
ax3.set_title('Monthly Funnel Performance Trends')
ax3.legend(bbox_to_anchor=(1.05, 1), loc='upper left')
ax3.grid(True, alpha=0.3)

# 4. Cohort-based Funnel
ax4 = axes[1, 1]

# Simulate cohort data
cohorts = ['Week 1', 'Week 2', 'Week 3', 'Week 4']
cohort_data = {
    'Sign-ups': [1000, 1100, 950, 1200],
    'Activated': [700, 770, 665, 840],
    'Day 7 Retention': [350, 385, 333, 420],
    'Day 30 Retention': [175, 193, 166, 210],
    'Converted': [87, 96, 83, 105]
}

cohort_df = pd.DataFrame(cohort_data, index=cohorts)

# Calculate retention rates for each cohort
retention_rates = cohort_df.div(cohort_df['Sign-ups'], axis=0) * 100

# Create heatmap-style visualization
im = ax4.imshow(retention_rates.values, cmap='RdYlBu_r', aspect='auto')

# Add text annotations
for i in range(len(cohorts)):
    for j in range(len(cohort_data.keys())):
        text = ax4.text(j, i, f'{retention_rates.iloc[i, j]:.1f}%',
                       ha="center", va="center", color="black", fontweight='bold')

ax4.set_xticks(range(len(cohort_data.keys())))
ax4.set_xticklabels(cohort_data.keys(), rotation=45, ha='right')
ax4.set_yticks(range(len(cohorts)))
ax4.set_yticklabels(cohorts)
ax4.set_title('Cohort-based Funnel Analysis\n(Retention Rates by Signup Week)')

plt.colorbar(im, ax=ax4, shrink=0.6, label='Retention Rate (%)')

plt.tight_layout()
plt.show()


In [ ]:
# Interactive funnel charts (Plotly structure)
print("Interactive Funnel Charts (Plotly):")
print("=" * 50)

print("\n1. Basic Interactive Funnel")
print("Code structure:")
print("""
fig = go.Figure(go.Funnel(
    y=df['Stage'],
    x=df['Count'],
    textinfo="value+percent initial+percent previous",
    textposition="inside",
    connector={"line": {"color": "royalblue", "dash": "solid", "width": 2}},
    marker={"color": ["deepskyblue", "lightsalmon", "tan", "teal", "silver"],
             "line": {"width": 2, "color": "wheat"}},
))

fig.update_layout(title="Interactive Sales Funnel")
fig.show()
""")

print("\n2. Multi-Level Funnel with Dropdowns")
print("Code structure:")
print("""
# Create traces for different funnels
fig = go.Figure()

for category in categories:
    category_data = df[df['Category'] == category]
    
    fig.add_trace(go.Funnel(
        y=category_data['Stage'],
        x=category_data['Count'],
        name=category,
        visible=True if category == categories[0] else False,
        textinfo="value+percent initial",
        connector={"line": {"color": "royalblue"}},
    ))

# Add dropdown menu
fig.update_layout(
    updatemenus=[
        dict(
            buttons=list([
                dict(label=cat,
                     method="update",
                     args=[{"visible": [c == cat for c in categories]}])
                for cat in categories
            ]),
            direction="down",
            showactive=True,
        )
    ]
)

fig.show()
""")

print("\n3. Animated Funnel Over Time")
print("Code structure:")
print("""
# Create animated funnel
fig = go.Figure()

for month in months:
    month_data = df[df['Month'] == month]
    
    fig.add_trace(go.Funnel(
        y=month_data['Stage'],
        x=month_data['Count'],
        name=month,
        visible=False,
        textinfo="value+percent initial"
    ))

# Set first frame visible
fig.data[0].visible = True

# Create animation frames
frames = []
for i, month in enumerate(months):
    frame = go.Frame(
        data=[go.Funnel(
            y=df[df['Month'] == month]['Stage'],
            x=df[df['Month'] == month]['Count'],
            textinfo="value+percent initial"
        )],
        name=month
    )
    frames.append(frame)

fig.frames = frames

# Add play/pause buttons
fig.update_layout(
    updatemenus=[
        dict(type="buttons",
             buttons=[dict(label="Play",
                          method="animate",
                          args=[None, {"frame": {"duration": 1000}}]),
                     dict(label="Pause",
                          method="animate",
                          args=[[None], {"frame": {"duration": 0}}])])
    ]
)

fig.show()
""")

print("\n4. Funnel with Custom Hover Information")
print("Code structure:")
print("""
# Add custom hover data
fig = go.Figure(go.Funnel(
    y=df['Stage'],
    x=df['Count'],
    text=df['Stage'],
    textinfo="text+value+percent initial",
    hovertemplate="<b>%{text}</b><br>" +
                  "Count: %{value}<br>" +
                  "Conversion Rate: %{percentInitial}<br>" +
                  "Step Conversion: %{percentPrevious}<br>" +
                  "<extra></extra>",
    connector={"line": {"color": "gray", "width": 1}},
    marker={"color": colors, "line": {"width": 1, "color": "white"}},
))

fig.update_layout(
    title="Sales Funnel with Detailed Hover Information",
    font_size=12
)

fig.show()
""")


In [ ]:
# Statistical analysis of funnel data
print("Funnel Chart Statistical Analysis:")
print("=" * 50)

# 1. Conversion Rate Analysis
print("1. CONVERSION RATE BENCHMARKING:")

funnels_analysis = {
    'E-commerce': ecommerce_funnel,
    'SaaS': saas_funnel,
    'Mobile App': app_funnel,
    'Recruitment': recruitment_funnel,
    'Healthcare': healthcare_funnel
}

conversion_summary = []

for funnel_name, funnel_data in funnels_analysis.items():
    overall_conversion = funnel_data['Conversion_Rate'].iloc[-1]
    avg_step_conversion = funnel_data['Step_Conversion'][1:].mean()  # Exclude first 100%
    worst_step = funnel_data['Step_Conversion'][1:].min()
    worst_stage_idx = funnel_data['Step_Conversion'][1:].idxmin()
    worst_stage = funnel_data['Stage'].iloc[worst_stage_idx]
    
    conversion_summary.append({
        'Funnel': funnel_name,
        'Overall_Conversion': overall_conversion,
        'Avg_Step_Conversion': avg_step_conversion,
        'Worst_Step_Conversion': worst_step,
        'Worst_Stage': worst_stage
    })
    
    print(f"\n   {funnel_name} Funnel:")
    print(f"     Overall Conversion: {overall_conversion:.2f}%")
    print(f"     Average Step Conversion: {avg_step_conversion:.2f}%")
    print(f"     Biggest Drop-off: {worst_stage} ({worst_step:.2f}%)")

conversion_df = pd.DataFrame(conversion_summary)

# Industry benchmarks comparison
print(f"\n   Funnel Performance Ranking (by overall conversion):")
ranked_funnels = conversion_df.sort_values('Overall_Conversion', ascending=False)
for i, row in ranked_funnels.iterrows():
    print(f"     {row['Funnel']}: {row['Overall_Conversion']:.2f}% overall")

# 2. Statistical Significance Testing
print(f"\n2. CONVERSION RATE STATISTICAL ANALYSIS:")

# Simulate confidence intervals for conversion rates
for funnel_name, funnel_data in funnels_analysis.items():
    initial_count = funnel_data['Count'].iloc[0]
    final_count = funnel_data['Count'].iloc[-1]
    
    # Calculate binomial confidence interval (Wilson score interval)
    conversion_rate = final_count / initial_count
    n = initial_count
    z = 1.96  # 95% confidence
    
    denominator = 1 + z**2/n
    center = (conversion_rate + z**2/(2*n)) / denominator
    half_width = z * np.sqrt((conversion_rate*(1-conversion_rate) + z**2/(4*n)) / n) / denominator
    
    ci_lower = (center - half_width) * 100
    ci_upper = (center + half_width) * 100
    
    print(f"   {funnel_name}: {conversion_rate*100:.2f}% (95% CI: {ci_lower:.2f}%-{ci_upper:.2f}%)")

# 3. Bottleneck Analysis
print(f"\n3. BOTTLENECK IDENTIFICATION:")

def identify_bottlenecks(funnel_data, threshold=70):
    """Identify stages with conversion rates below threshold"""
    bottlenecks = []
    step_conversions = funnel_data['Step_Conversion'][1:]  # Exclude first stage
    
    for idx, conversion in step_conversions.items():
        if conversion < threshold:
            stage = funnel_data['Stage'].iloc[idx]
            bottlenecks.append((stage, conversion))
    
    return bottlenecks

for funnel_name, funnel_data in funnels_analysis.items():
    bottlenecks = identify_bottlenecks(funnel_data, threshold=70)
    
    print(f"\n   {funnel_name} Bottlenecks (<70% step conversion):")
    if bottlenecks:
        for stage, conversion in bottlenecks:
            severity = "Critical" if conversion < 50 else "Moderate" if conversion < 60 else "Minor"
            print(f"     {stage}: {conversion:.1f}% ({severity})")
    else:
        print(f"     No significant bottlenecks identified")

# 4. Revenue Impact Analysis
print(f"\n4. REVENUE IMPACT ANALYSIS:")

# Simulate revenue per conversion
revenue_assumptions = {
    'E-commerce': {'avg_order_value': 75, 'profit_margin': 0.25},
    'SaaS': {'avg_order_value': 99, 'profit_margin': 0.80},
    'Mobile App': {'avg_order_value': 4.99, 'profit_margin': 0.70},
    'Recruitment': {'avg_order_value': 15000, 'profit_margin': 0.15},
    'Healthcare': {'avg_order_value': 250, 'profit_margin': 0.40}
}

print("   Revenue Impact of Improving Conversion Rates:")
for funnel_name, funnel_data in funnels_analysis.items():
    if funnel_name in revenue_assumptions:
        assumptions = revenue_assumptions[funnel_name]
        
        initial_count = funnel_data['Count'].iloc[0]
        final_count = funnel_data['Count'].iloc[-1]
        current_revenue = final_count * assumptions['avg_order_value'] * assumptions['profit_margin']
        
        # Calculate impact of 10% improvement in overall conversion
        improved_conversions = final_count * 1.1
        improved_revenue = improved_conversions * assumptions['avg_order_value'] * assumptions['profit_margin']
        revenue_lift = improved_revenue - current_revenue
        
        print(f"\n     {funnel_name}:")
        print(f"       Current Revenue: ${current_revenue:,.0f}")
        print(f"       10% Conversion Improvement: +${revenue_lift:,.0f}")
        
        # Identify highest impact stage to improve
        bottlenecks = identify_bottlenecks(funnel_data, threshold=80)
        if bottlenecks:
            worst_stage, worst_conversion = min(bottlenecks, key=lambda x: x[1])
            stage_idx = funnel_data[funnel_data['Stage'] == worst_stage].index[0]
            
            # Calculate potential impact of fixing this stage
            current_stage_count = funnel_data['Count'].iloc[stage_idx-1]
            if worst_conversion < 80:  # If there's room for improvement
                improved_stage_conversion = min(80, worst_conversion + 15)  # Improve by 15pp or to 80%
                additional_conversions = current_stage_count * (improved_stage_conversion - worst_conversion) / 100
                stage_revenue_impact = additional_conversions * assumptions['avg_order_value'] * assumptions['profit_margin']
                
                print(f"       Fixing '{worst_stage}': +${stage_revenue_impact:,.0f} potential")

# 5. Cohort and Segmentation Analysis
print(f"\n5. CAMPAIGN FUNNEL COMPARISON:")

# Analyze campaign performance
campaign_performance = []
for campaign in campaign_names:
    campaign_data = campaigns_df[campaigns_df['Campaign'] == campaign]
    impressions = campaign_data[campaign_data['Stage'] == 'Impressions']['Count'].iloc[0]
    customers = campaign_data[campaign_data['Stage'] == 'Customers']['Count'].iloc[0]
    
    overall_conversion = (customers / impressions) * 100
    ctr = (campaign_data[campaign_data['Stage'] == 'Clicks']['Count'].iloc[0] / impressions) * 100
    lead_conversion = (campaign_data[campaign_data['Stage'] == 'Leads']['Count'].iloc[0] / 
                      campaign_data[campaign_data['Stage'] == 'Clicks']['Count'].iloc[0]) * 100
    
    campaign_performance.append({
        'Campaign': campaign,
        'Overall_Conversion': overall_conversion,
        'CTR': ctr,
        'Lead_Conversion': lead_conversion,
        'Total_Customers': customers
    })

campaign_df = pd.DataFrame(campaign_performance)
campaign_df = campaign_df.sort_values('Overall_Conversion', ascending=False)

print("   Marketing Campaign Performance:")
for _, row in campaign_df.iterrows():
    print(f"     {row['Campaign']}: {row['Overall_Conversion']:.3f}% overall, {row['Total_Customers']} customers")

# Calculate statistical significance between campaigns
best_campaign = campaign_df.iloc[0]
worst_campaign = campaign_df.iloc[-1]

print(f"\n   Performance Gap Analysis:")
print(f"     Best: {best_campaign['Campaign']} ({best_campaign['Overall_Conversion']:.3f}%)")
print(f"     Worst: {worst_campaign['Campaign']} ({worst_campaign['Overall_Conversion']:.3f}%)")
print(f"     Performance Gap: {best_campaign['Overall_Conversion'] - worst_campaign['Overall_Conversion']:.3f} percentage points")

# 6. Funnel Optimization Recommendations
print(f"\n6. FUNNEL OPTIMIZATION RECOMMENDATIONS:")

print("   General Best Practices:")
print("   ✓ Focus on stages with <70% conversion rates first")
print("   ✓ A/B test high-impact, low-conversion stages")
print("   ✓ Monitor funnel performance weekly")
print("   ✓ Segment funnels by traffic source/customer type")
print("   ✓ Set up conversion tracking for each stage")
print("   ✓ Calculate customer lifetime value for ROI analysis")

print(f"\n   Visualization Guidelines:")
print("   ✓ Use consistent colors across funnel stages")
print("   ✓ Show both counts and percentages")
print("   ✓ Highlight biggest drop-off points")
print("   ✓ Include step-by-step conversion rates")
print("   ✓ Compare funnels side-by-side when relevant")
print("   ✓ Add time-based analysis for trends")

print(f"\nWhen to Use Funnel Charts:")
print("   • Sales process optimization")
print("   • User journey analysis")
print("   • Marketing campaign evaluation")
print("   • Conversion rate optimization")
print("   • Process efficiency measurement")
print("   • Customer onboarding analysis")

print(f"\nAlternatives to Consider:")
print("   • Sankey diagrams for complex flow visualization")
print("   • Waterfall charts for step-by-step changes")
print("   • Bar charts for simple conversion comparison")
print("   • Cohort analysis for retention over time")
